In [1]:
from matplotlib import pyplot as plt
import numpy as np 
import pandas as pd 
import os
from tqdm import tqdm
day = 10

In [9]:
feat_list = [
            'open','high','low','close','vwap','volume','pct_chg','turnover','free_turnover'
#             "open_ts_norm_30d","high_ts_norm_30d","low_ts_norm_30d","close_ts_norm_30d","vwap_ts_norm_30d" 
#             ,"volume_ts_norm_30d" ,"pct_chg_ts_norm_30d" ,"turnover_ts_norm_30d" ,"free_turnover_ts_norm_30d" 
]

In [2]:
dict_load = np.load("/home/wuwenjun/jupyter_code/Shannon/AlphaNet/Factor_Description/base_list.npy",allow_pickle=True)
dict_load = dict_load.item()
dict_load

{'open': 'sub(open,0)',
 'high': 'sub(high,0)',
 'low': 'sub(low,0)',
 'close': 'sub(close,0)',
 'vwap': 'sub(vwap,0)',
 'volume': 'sub(volume,0)',
 'pct_chg': 'sub(pct_chg,0)',
 'turnover': 'sub(turnover,0)',
 'free_turnover': 'sub(free_turnover,0)',
 'open_ts_norm_30d': 'div(sub(open,mavg(30,open)),mstd(30,open))',
 'high_ts_norm_30d': 'div(sub(high,mavg(30,high)),mstd(30,high))',
 'low_ts_norm_30d': 'div(sub(low,mavg(30,low)),mstd(30,low))',
 'close_ts_norm_30d': 'div(sub(close,mavg(30,close)),mstd(30,close))',
 'vwap_ts_norm_30d': 'div(sub(vwap,mavg(30,vwap)),mstd(30,vwap))',
 'volume_ts_norm_30d': 'div(sub(volume,mavg(30,volume)),mstd(30,volume))',
 'pct_chg_ts_norm_30d': 'div(sub(pct_chg,mavg(30,pct_chg)),mstd(30,pct_chg))',
 'turnover_ts_norm_30d': 'div(sub(turnover,mavg(30,turnover)),mstd(30,turnover))',
 'free_turnover_ts_norm_30d': 'div(sub(free_turnover,mavg(30,free_turnover)),mstd(30,free_turnover))'}

In [12]:
# np.save("/home/ShareFolder/feature_platform/ti0/wuwenjun/1Factor_Description/base_list.npy",dict_load)
# np.save("/home/wuwenjun/jupyter_code/Shannon/AlphaNet/Factor_Description/base_list.npy",dict_load)

In [3]:
name_list = []
task_list = []
for i in dict_load.keys():
    name_list.append(i)
    task_list.append(dict_load[i])

In [58]:
name_list = []
task_list = []
# Moving Cov
for i in range(len(feat_list)-1):
    for j in range(i+1,len(feat_list)):
        name_list.append("COV_%s_%s_%s" % (feat_list[i],feat_list[j],day))
        task_list.append("mcovar(%s,%s,%s)" % (day,dict_load[feat_list[i]],dict_load[feat_list[j]]))

# Moving Corr
for i in range(len(feat_list)-1):
    for j in range(i+1,len(feat_list)):
        name_list.append("CORR_%s_%s_%s" % (feat_list[i],feat_list[j],day))
        task_list.append("ts_corr(%s,%s,%s)" % (day,dict_load[feat_list[i]],dict_load[feat_list[j]]))

# # Moving STD
for i in range(len(feat_list)-1):
    name_list.append("STD_%s_%s" % (feat_list[i],day))
    task_list.append("mstd(%s,%s)" % (day,dict_load[feat_list[i]]))
    
# # Z-score
for i in range(len(feat_list)-1):
    name_list.append("ZSCORE_%s_%s" % (feat_list[i],day)) 
    task_list.append("div(mavg(%s,%s),mstd(%s,%s))" % (day,dict_load[feat_list[i]],day,dict_load[feat_list[i]]))
    
# # Return
for i in range(len(feat_list)-1):
    name_list.append("RETURN_%s_%s" % (feat_list[i],day))
    task_list.append("div(sub(%s,delay(%s,%s)),delay(%s,%s))" % (dict_load[feat_list[i]],day,dict_load[feat_list[i]],day,dict_load[feat_list[i]])) 
# Decay 
for i in range(len(feat_list)-1):
    name_list.append("DECAY_%s_%s" % (feat_list[i],day))
    task_list.append("decay(%s,%s)" % (day,dict_load[feat_list[i]])

In [4]:
final = ""
for i in name_list:
    final += i + ";"
print("name")
print(final)

name
open;high;low;close;vwap;volume;pct_chg;turnover;free_turnover;open_ts_norm_30d;high_ts_norm_30d;low_ts_norm_30d;close_ts_norm_30d;vwap_ts_norm_30d;volume_ts_norm_30d;pct_chg_ts_norm_30d;turnover_ts_norm_30d;free_turnover_ts_norm_30d;


In [5]:
final = ""
for i in task_list:
    final += i + ";"
print("task")
print(final)

task
sub(open,0);sub(high,0);sub(low,0);sub(close,0);sub(vwap,0);sub(volume,0);sub(pct_chg,0);sub(turnover,0);sub(free_turnover,0);div(sub(open,mavg(30,open)),mstd(30,open));div(sub(high,mavg(30,high)),mstd(30,high));div(sub(low,mavg(30,low)),mstd(30,low));div(sub(close,mavg(30,close)),mstd(30,close));div(sub(vwap,mavg(30,vwap)),mstd(30,vwap));div(sub(volume,mavg(30,volume)),mstd(30,volume));div(sub(pct_chg,mavg(30,pct_chg)),mstd(30,pct_chg));div(sub(turnover,mavg(30,turnover)),mstd(30,turnover));div(sub(free_turnover,mavg(30,free_turnover)),mstd(30,free_turnover));


In [7]:
alpha_list = np.load("/home/wuwenjun/jupyter_code/Shannon/AlphaNet/Factor_Description/feature_list.npy",allow_pickle=True)
alpha_list = alpha_list.item()
Cover = False
for i in range(len(task_list)):
    if name_list[i] in alpha_list and alpha_list[name_list[i]] != task_list[i] and Cover == False:
        raise ValueError("contain " + name_list[i])
    alpha_list[name_list[i]] = task_list[i]
check_list = list(os.listdir("/home/ShareFolder/feature_platform/ti0/wuwenjun"))
for i in alpha_list.keys():
    if i not in check_list:
        raise ValueError("No [%s] in feature_platform" %i)
alpha_list

{'COV_open_high_10': 'mcovar(10,open,high)',
 'COV_open_low_10': 'mcovar(10,open,low)',
 'COV_open_close_10': 'mcovar(10,open,close)',
 'COV_open_vwap_10': 'mcovar(10,open,vwap)',
 'COV_open_volume_10': 'mcovar(10,open,volume)',
 'COV_open_pct_chg_10': 'mcovar(10,open,pct_chg)',
 'COV_open_turnover_10': 'mcovar(10,open,turnover)',
 'COV_open_free_turnover_10': 'mcovar(10,open,free_turnover)',
 'COV_high_low_10': 'mcovar(10,high,low)',
 'COV_high_close_10': 'mcovar(10,high,close)',
 'COV_high_vwap_10': 'mcovar(10,high,vwap)',
 'COV_high_volume_10': 'mcovar(10,high,volume)',
 'COV_high_pct_chg_10': 'mcovar(10,high,pct_chg)',
 'COV_high_turnover_10': 'mcovar(10,high,turnover)',
 'COV_high_free_turnover_10': 'mcovar(10,high,free_turnover)',
 'COV_low_close_10': 'mcovar(10,low,close)',
 'COV_low_vwap_10': 'mcovar(10,low,vwap)',
 'COV_low_volume_10': 'mcovar(10,low,volume)',
 'COV_low_pct_chg_10': 'mcovar(10,low,pct_chg)',
 'COV_low_turnover_10': 'mcovar(10,low,turnover)',
 'COV_low_free_tur

In [8]:
np.save("/home/wuwenjun/jupyter_code/Shannon/AlphaNet/Factor_Description/feature_list.npy",alpha_list)
np.save("/home/ShareFolder/feature_platform/ti0/wuwenjun/1Factor_Description/feature_list.npy",alpha_list)